# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [ ]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders` from [here](https://drive.google.com/file/d/17oJ9NNbwJ8Wv4Jp-mLFiC59k72Brdyha/view?usp=sharing) and place it in the provided data folder. You will access the data folder with a relative path.
Load `Orders` into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [ ]:
# your code here
orders = pd.read_csv('./data/Orders.csv', index_col = 0)
orders.head()

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [ ]:
# your code here

# select 'CustomerID', and 'amount_spent', and sum per customer 
cust_amount = orders[['CustomerID', 'amount_spent']]

# sum amount spent per customer
cust_total = cust_amount.groupby('CustomerID').sum()

# rename column with summed total per customer to 'amount_total'
cust_total = cust_total.rename(columns={'amount_spent':'amount_total'})

cust_total.head()

In [ ]:
# Calculate 0.75 and 0.95 quantiles

Q95 = cust_total['amount_total'].quantile(.95)
Q75 = cust_total['amount_total'].quantile(.75)

# Add customer labels to new column, based on in which quantile their spending falls
cust_total.loc[(cust_total['amount_total'] <= Q75), 'customer_status'] = 'Normal'
cust_total.loc[(cust_total['amount_total'] > Q75) & (cust_total['amount_total'] <= Q95), 'customer_status'] = 'Preferred'
cust_total.loc[(cust_total['amount_total'] > Q95), 'customer_status'] = 'VIP'

# Check dataframe
cust_total.head(2)


In [ ]:
# Add customer labels to original dataset
orders_updated = pd.merge(orders, cust_total, on='CustomerID')

#Check updated orders dataframe
orders_updated.head()

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [ ]:
# your code here

# select Customer ID, Country and Customer status columns.
selection = orders_updated[['CustomerID', 'Country', 'customer_status']]

# select only the rows that have VIP status
selection_VIP = selection[selection.customer_status == 'VIP']

# group by country, count VIP customer IDs for each country 
VIP_per_country = selection_VIP.groupby('Country')[['CustomerID']].count()

# rename column
VIP_per_country = VIP_per_country.rename(columns={'CustomerID':'VIP'})

# sort values from large to small to find Country with most VIP customers
VIP_per_country.sort_values('VIP', ascending = False)

# country with the most VIP customers is the United Kingdom


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [ ]:
# your code here

# same as above, select the rows with Preferred status
selection_preferred = selection[selection.customer_status == 'Preferred']

# group by country, count preferred customer IDs for each country 
preferred_per_country = selection_preferred.groupby('Country')[['CustomerID']].count()

# rename column
preferred_per_country = preferred_per_country.rename(columns={'CustomerID':'Preferred'})

# merge VIP and preferred counts per country with an outer join 
pref_VIP_country = pd.merge(preferred_per_country, VIP_per_country, on='Country', how = 'outer')

# Create new column to sum the number of preferred and VIP customers per country
pref_VIP_country['Total'] = pref_VIP_country['Preferred'] + pref_VIP_country['VIP']

# sort values from large to small to find Country with most preferred + VIP customers combined
pref_VIP_country.sort_values('Total', ascending = False)

# country with the most preferred+VIP customers combined is the United Kingdom